In [1]:
import importlib, sys
# script 경로 window용
sys.path.append(r"C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts") 
from glob import glob
import pandas as pd
import conn_db
import helper, trade_cleaner

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)  

In [2]:
all_files = glob(conn_db.get_path('신성질별_수출입_raw') + "*.xls")
df = pd.concat([pd.read_excel(file, skiprows=4)
               for file in all_files], axis=0).drop_duplicates()

C:\Users\bong2\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df.head(3)

,기간,수출입구분,성질명,중량,금액
0,총계,NaN,NaN,"372,738,213.9","152,103,189"
1,2002.01,수입,1.소비재,"1,219,724.9","1,345,730"
2,2002.01,수입,가.직접소비재,"1,136,886.8","669,369"


In [4]:
filt1 = df['성질명'].str.startswith('(')
filt2 = df['성질명'].str.endswith(')')
filt3 = df['기간'] != '총계'
filt = filt1 & filt2 & filt3
df = df.loc[filt].reset_index(drop=True)

In [5]:
df.head()

,기간,수출입구분,성질명,중량,금액
0,2002.01,수입,(쌀),"22,538.8","5,269"
1,2002.01,수입,(밀),"166,087.6","27,796"
2,2002.01,수입,(보리),"28,220.5","5,075"
3,2002.01,수입,(옥수수),"164,693.1","18,983"
4,2002.01,수입,(대두),"95,152.1","18,843"


In [6]:
df = helper.drop_column(df, 'Unnamed')

In [7]:
df['중량'] = pd.to_numeric(df['중량'].str.replace(',', ''))
df['금액'] = pd.to_numeric(df['금액'].str.replace(',', ''))
df.rename(columns={'기간': '날짜'}, inplace=True)
df['날짜'] = pd.to_datetime(df['날짜'])

In [8]:
trade_types = ['수출','수입']
result_df = pd.DataFrame()
for trade_type in trade_types:
    temp = df.loc[df['수출입구분'] == trade_type].copy()
    temp.rename(columns={'중량': trade_type+'중량 (ton)',
                         '금액': trade_type+'금액 (천$)'}, inplace=True)
    temp.drop(columns='수출입구분', inplace=True)
    result_df = result_df.append(temp)

result_df.reset_index(drop=True, inplace=True)
result_df = result_df.groupby(['날짜', '성질명']).agg(sum).reset_index()
result_df['성질명'] = result_df['성질명'].str[1:-1].str.strip()

In [9]:
result_df.head(3)

,날짜,성질명,수출중량 (ton),수출금액 (천$),수입중량 (ton),수입금액 (천$)
0,2002-01-01,CRT TV,21.3,452.0,5.3,767.0
1,2002-01-01,LCD TV,1334.8,10312.0,2.5,58.0
2,2002-01-01,PDP TV,431.1,5487.0,4.9,203.0


In [10]:
# 맵핑
map_df = conn_db.from_('Master_수출입품목', '신성질별_short')

df = map_df.merge(result_df, left_on='세세분류명', right_on='성질명',
                  how='inner').drop(columns='성질명')

In [11]:
df.head(3)

,대분류명,중분류명,소분류명,세분류명,세세분류명,설명,날짜,수출중량 (ton),수출금액 (천$),수입중량 (ton),수입금액 (천$)
0,소비재,간이세율적용분,간이세율적용분,간이세율적용분,간이세율적용분,이사화물,2002-01-01,2010.7,2782.0,1153.4,613.0
1,소비재,간이세율적용분,간이세율적용분,간이세율적용분,간이세율적용분,이사화물,2002-02-01,2332.3,3076.0,1301.5,491.0
2,소비재,간이세율적용분,간이세율적용분,간이세율적용분,간이세율적용분,이사화물,2002-03-01,2256.3,4096.0,1800.8,632.0


In [12]:
df = trade_cleaner.add_units(df)

In [13]:
[df.rename(columns={col: col[:-1]}, inplace=True) for col in df.columns.tolist() if '분류명' in col]

[None, None, None, None, None]

In [14]:
df.head(3)

,대분류,중분류,소분류,세분류,세세분류,설명,날짜,수출중량 (ton),수출금액 (천$),수입중량 (ton),수입금액 (천$),수출금액 (백만$),수입금액 (백만$),무역규모 (백만$),무역수지 (백만$),수출금액 (억$),수입금액 (억$),무역규모 (억$),무역수지 (억$),무역수지 (천$),무역규모 (천$),수출ton당 (천$/ton),수입ton당 (천$/ton)
0,소비재,간이세율적용분,간이세율적용분,간이세율적용분,간이세율적용분,이사화물,2002-01-01,2010.7,2782.0,1153.4,613.0,2.782,0.613,3.395,2.169,0.02782,0.00613,0.03395,0.02169,2169.0,3395.0,1.383598,0.531472
1,소비재,간이세율적용분,간이세율적용분,간이세율적용분,간이세율적용분,이사화물,2002-02-01,2332.3,3076.0,1301.5,491.0,3.076,0.491,3.567,2.585,0.03076,0.00491,0.03567,0.02585,2585.0,3567.0,1.318870,0.377257
2,소비재,간이세율적용분,간이세율적용분,간이세율적용분,간이세율적용분,이사화물,2002-03-01,2256.3,4096.0,1800.8,632.0,4.096,0.632,4.728,3.464,0.04096,0.00632,0.04728,0.03464,3464.0,4728.0,1.815361,0.350955


In [15]:
cols_long = ['대분류', '중분류','소분류','세분류','세세분류','설명','날짜',
            '수입중량 (ton)','수입금액 (천$)', '수출중량 (ton)', '수출금액 (천$)',
            '수입금액 (백만$)', '수출금액 (백만$)', '무역규모 (백만$)', '무역수지 (백만$)',
            '수입금액 (억$)', '수출금액 (억$)', '무역규모 (억$)', '무역수지 (억$)', '무역수지 (천$)',
            '무역규모 (천$)', '수출ton당 (천$/ton)', '수입ton당 (천$/ton)']

cols_short = ['날짜','대분류', '중분류','소분류','세분류','세세분류','설명',
              '수출중량 (ton)','수입중량 (ton)','수입금액 (천$)',  '수출금액 (천$)',
              '무역수지 (천$)','무역규모 (천$)'] 

In [16]:
# 저장
conn_db.export_(df[cols_long], '수출입_신성질별')
for sheet in ['DB_수출입_신성질별','수출입_신성질별_대시보드']:
    conn_db.to_(df[cols_short], sheet, 'import')

C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts\conn_db.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all', inplace=True)
C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts\conn_db.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all', inplace=True)
